In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import h5py

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

In [2]:
data_path2 = "../FST/eval_full/sah_full.dev"

cols =["LEMMA", "WORD", "POS"]
import pandas as pd
df = pd.read_csv(data_path2, sep="\t", names = cols, encoding="utf-8")

In [3]:
df.LEMMA = df.LEMMA + df.POS
df = df[["LEMMA", "WORD", "POS"]]

In [4]:
df = df.drop("POS", axis=1)
df

,LEMMA,WORD
0,бу<DET>,бу
1,аайы<DET>,аайы
2,бу<DET>|<Case=Acc>,маны
3,яблоко<NOUN>|<Number=Sing>|<Case=Abl>,яблокаттан
4,ыстаал<NOUN>|<Number=Sing>|<Case=Abl>,ыстаалтан
...,...,...
664,ыҥыр<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,ыҥырар
665,гын<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,гынар
666,уст<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,устар
667,күл<VERB>|<Tense=Pres>|<VerbForm=Trans>,күлэ


In [5]:
batch_size = 32  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = len(df)  # Number of samples to train on.

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(df["WORD"].to_list(), df["LEMMA"].to_list(), train_size=0.8, shuffle=True)

In [7]:
train = list(zip(X_train,y_train))
train = pd.DataFrame(train, columns=['FORM','POS'])
train.to_csv("train.dev", encoding="utf-8", sep="\t", index=False, header=False)

In [8]:
valid = list(zip(X_valid,y_valid))
valid = pd.DataFrame(valid, columns=['FORM','POS'])
valid.to_csv("valid.test", encoding="utf-8", sep="\t", index=False, header=False)

In [9]:
train

,FORM,POS
0,бэс,бэс<NOUN>|<Number=Sing>|<Case=Nom>
1,үөрэтэр,үөрэт<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
2,үрдүкү,үрдүкү<ADJ>
3,сыалыһар,сыалыһар<NOUN>|<Number=Sing>|<Case=Nom>
4,оҕо,оҕо<NOUN>|<Number=Sing>|<Case=Nom>
...,...,...
530,араас,араас<ADV>
531,иккитэ,икки<NUM>|<Case=Par>|<NumType=Card>
532,турар,тур<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
533,онуоха,ол<PRON>|<Case=Dat>|<PronType=Int>


In [10]:
valid

,FORM,POS
0,олоппосторго,олоппос<NOUN>|<Number=Plur>|<Case=Dat>
1,бэргэһэ,бэргэһэ<NOUN>|<Number=Sing>|<Case=Nom>
2,оҕуруот,оҕуруот<NOUN>|<Number=Sing>|<Case=Nom>
3,ууһа,уус<NOUN>|<Number=Sing>|<Number[psor]=Sing>|<P...
4,хайаҕа,хайа<NOUN>|<Number=Sing>|<Case=Dat>
...,...,...
129,айымньыта,айымньы<NOUN>|<Number=Sing>|<Case=Par>
130,кэтэҕин,кэт<VERB>|<Number=Sing>|<Person=2>|<Tense=Pres>
131,үөрэнэҕин,үөрэн<VERB>|<Number=Sing>|<Person=2>|<Tense=Pres>
132,үөрэнэр,үөрэн<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>


In [11]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open("train.dev", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text  = line.split('\t')
    #print(input_text)
    
    #x_train, x_valid, y_train, y_valid = train_test_split(input_texts, target_texts, test_size=0.33, shuffle= True)

    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    #target_text = target_text# + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 535
Number of unique input tokens: 32
Number of unique output tokens: 84
Max sequence length for inputs: 14
Max sequence length for outputs: 80


In [12]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


In [13]:
#test 

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[char]] = 1.
    decoder_target_data[i, t:, target_token_index[char]] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
14/14 [==============================] - 3s 123ms/step - loss: 2.7452 - accuracy: 0.4820 - val_loss: 2.3806 - val_accuracy: 0.4815
Epoch 2/100
14/14 [==============================] - 1s 103ms/step - loss: 2.2145 - accuracy: 0.5185 - val_loss: 2.2069 - val_accuracy: 0.4938
Epoch 3/100
14/14 [==============================] - 1s 105ms/step - loss: 2.1136 - accuracy: 0.5372 - val_loss: 2.0655 - val_accuracy: 0.5063
Epoch 4/100
14/14 [==============================] - 1s 100ms/step - loss: 1.9497 - accuracy: 0.5404 - val_loss: 1.8819 - val_accuracy: 0.5072
Epoch 5/100
14/14 [==============================] - 2s 109ms/step - loss: 1.7724 - accuracy: 0.5500 - val_loss: 1.8699 - val_accuracy: 0.5111
Epoch 6/100
14/14 [==============================] - 1s 102ms/step - loss: 1.6268 - accuracy: 0.5733 - val_loss: 1.6357 - val_accuracy: 0.5324
Epoch 7/100
14/14 [==============================] - 2s 111ms/step - loss: 1.5375 - accuracy: 0.6001 - val_loss: 1.5608 - val_accuracy: 0.5787

Epoch 58/100
14/14 [==============================] - 2s 120ms/step - loss: 0.0781 - accuracy: 0.9764 - val_loss: 0.1763 - val_accuracy: 0.9491
Epoch 59/100
14/14 [==============================] - 2s 122ms/step - loss: 0.0800 - accuracy: 0.9763 - val_loss: 0.1739 - val_accuracy: 0.9495
Epoch 60/100
14/14 [==============================] - 2s 124ms/step - loss: 0.0735 - accuracy: 0.9779 - val_loss: 0.1730 - val_accuracy: 0.9494
Epoch 61/100
14/14 [==============================] - 2s 121ms/step - loss: 0.0791 - accuracy: 0.9770 - val_loss: 0.1708 - val_accuracy: 0.9513
Epoch 62/100
14/14 [==============================] - 2s 118ms/step - loss: 0.0669 - accuracy: 0.9811 - val_loss: 0.1725 - val_accuracy: 0.9521
Epoch 63/100
14/14 [==============================] - 2s 121ms/step - loss: 0.0642 - accuracy: 0.9814 - val_loss: 0.1845 - val_accuracy: 0.9486
Epoch 64/100
14/14 [==============================] - 2s 122ms/step - loss: 0.0666 - accuracy: 0.9794 - val_loss: 0.1717 - val_accuracy:

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 32)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 84)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        295936      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [16]:
# Save model
model.save('sah.h5')

In [17]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: бэс
Decoded sentence: бэс<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: үөрэтэр
Decoded sentence: үөрэт<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>

-
Input sentence: үрдүкү
Decoded sentence: үрдүкү<ADJ>

-
Input sentence: сыалыһар
Decoded sentence: сыалыһар<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: оҕо
Decoded sentence: оҕо<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: аҕаһым
Decoded sentence: аҕас<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>

-
Input sentence: ааҕар
Decoded sentence: аах<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>

-
Input sentence: үрүҥ
Decoded sentence: үрүҥ<ADJ>

-
Input sentence: көрбөт
Decoded sentence: көр<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>

-
Input sentence: бурдугу
Decoded sentence: бурдук<NOUN>|<Number=Sing>|<Case=Nom>



In [20]:
val_input_texts = [] # valid["FORM"].to_list()
val_target_texts = [] # valid["POS"].to_list()
line_ix = len(valid)

with open("valid.test", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text  = line.split('\t')
        val_input_texts.append(input_text)
        val_target_texts.append(target_text)

val_encoder_input_data = np.zeros((len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

try:
    for i, input_text in enumerate(val_input_texts):
        for t, char in enumerate(input_text):
        #print(val_encoder_input_data[i,t])
            val_encoder_input_data[i, t, input_token_index[char]] = 1.

except KeyError:
    pass

In [21]:
y_true1 = []
y_pred1 = []

In [22]:
for seq_index in range(len(val_input_texts)):
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    y_pred1.append(decoded_sentence[:-1])
    y_true1.append(val_target_texts[seq_index])
    
    print('-')
    print('Input word:', val_input_texts[seq_index])
    print('Decoded feature:', decoded_sentence[:-1])
    print('Correct feature:', val_target_texts[seq_index])

-
Input word: олоппосторго
Decoded feature: олоппос<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: олоппос<NOUN>|<Number=Plur>|<Case=Dat>
-
Input word: бэргэһэ
Decoded feature: бэргэһэ<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: бэргэһэ<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: оҕуруот
Decoded feature: оҕо<NOUN>|<Number=Sing>|<Case=Acc>
Correct feature: оҕуруот<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: ууһа
Decoded feature: у<NOUN>|<Number=Sing>|<Number[psor]=Sing>|<Person[psor]=1>|<Case=Nom>
Correct feature: уус<NOUN>|<Number=Sing>|<Number[psor]=Sing>|<Person[psor]=3>|<Case=Nom>
-
Input word: хайаҕа
Decoded feature: хаха<PROPN>|<Case=Dat>
Correct feature: хайа<NOUN>|<Number=Sing>|<Case=Dat>
-
Input word: уута
Decoded feature: утуй<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: уу<NOUN>|<Number=Sing>|<Case=Par>
-
Input word: кэл
Decoded feature: кэл<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: кэл<VERB>|<Mood=Imp>|<Number=Sing>|<Person=2

-
Input word: балык
Decoded feature: балыктаа<VERB>|<Number=Plur>|<Person=3>|<Tense=Pres>
Correct feature: балык<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: халтархай
Decoded feature: халаан<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: халтархай<ADJ>
-
Input word: коперник
Decoded feature: фортока<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: коперник<PROPN>
-
Input word: айаана
Decoded feature: аа<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: айаана<PROPN>
-
Input word: эһиги
Decoded feature: иһиг<PRON>|<Case=Dat>
Correct feature: эһиги<PRON>|<Case=Nom>|<Number=Plur>|<Person=2>|<PronType=Prs>
-
Input word: киһи
Decoded feature: кии<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: киһи<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: онус
Decoded feature: онон<ADV>
Correct feature: онус<NUM>|<NumType=Card>
-
Input word: чаанньык
Decoded feature: чаан<ADJ>
Correct feature: чаанньык<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: туох
Decoded feature: тот<VERB>|<Number=S

-
Input word: тоҥор
Decoded feature: оттон<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: тоҥ<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
-
Input word: ортотугар
Decoded feature: орор<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: ортотугар<ADP>
-
Input word: собо
Decoded feature: сосоол<NOUN>|<Number=Sing>|<Case=Nom>
Correct feature: собо<NOUN>|<Number=Sing>|<Case=Nom>
-
Input word: килиэптэ
Decoded feature: киэ<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: килиэп<NOUN>|<Number=Sing>|<Case=Par>
-
Input word: барар
Decoded feature: бар<VERB>|<Int=Yes>|<Number=Sing>|<Person=3>|<Tense=Pres>
Correct feature: бар<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
-
Input word: ылыаҕым
Decoded feature: ыйаа<VERB>|<Number=Plur>|<Person=3>|<Tense=NearPast>
Correct feature: ыл<VERB>|<Number=Sing>|<Person=1>|<Tense=Fut>
-
Input word: ханнык
Decoded feature: ханн<ADV>
Correct feature: ханнык<PRON>|<PronType=Int>
-
Input word: олоҥхо
Decoded feature: олоп<VERB>|<Num

In [23]:
y_true = np.array(y_true1)
y_pred = np.array(y_pred1)

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [39]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true, y_pred, average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true, y_pred, average='weighted')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true, y_pred, average='weighted')
print('F1: %f' % f1)

Accuracy: 0.029851
Precision: 0.026119
Recall: 0.029851
F1: 0.027363


/opt/homebrew/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
